## Loading the Dataset

In [1]:
# Standard includes
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Useful module for dealing with the Gaussian density
from scipy.stats import norm, multivariate_normal 
# installing packages for interactive graphs
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider

In [2]:
# Load data set.
data = np.loadtxt('../Dataset/wine.data', delimiter=',')
# Names of features
featurenames = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash','Magnesium', 'Total phenols', 
                'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 
                'OD280/OD315 of diluted wines', 'Proline']
# Split 178 instances into training set (trainx, trainy) of size 130 and test set (testx, testy) of size 48
np.random.seed(0)
perm = np.random.permutation(178)
trainx = data[perm[0:130],1:14]
trainy = data[perm[0:130],0]
testx = data[perm[130:178], 1:14]
testy = data[perm[130:178],0]

## Fitting the Gaussian Generative Model

In [3]:
def fit_generative_model(x,y):
    k = 3  # labels 1,2,...,k
    d = (x.shape)[1]  # number of features
    mu = np.zeros((k+1,d))
    sigma = np.zeros((k+1,d,d))
    pi = np.zeros(k+1)
    for label in range(1,k+1):
        indices = (y == label)
        mu[label] = np.mean(x[indices,:], axis=0)
        sigma[label] = np.cov(x[indices,:], rowvar=0, bias=1)
        pi[label] = float(sum(indices))/float(len(y))
    return mu, sigma, pi

In [4]:
# Fit a Gaussian generative model to the training data
mu, sigma, pi = fit_generative_model(trainx,trainy)

## Testing the Model

In [5]:
# Now test the performance of a predictor based on a subset of features
def test_model(mu, sigma, pi, features, tx, ty):
    features=list(set(features))
    k = 3 # Labels 1,2,...,k
    nt = len(ty) # Number of test points
    score = np.zeros((nt,k+1))
    for i in range(0,nt):
        for label in range(1,k+1):
            sig1=sigma[label,features,:]
            score[i,label] = np.log(pi[label]) + \
            multivariate_normal.logpdf(tx[i,features], mean=mu[label,features], cov=sig1[:,features])
    predictions = np.argmax(score[:,1:4], axis=1) + 1
    # Finally, tally up score
    errors = np.sum(predictions != ty)
    print("Test error using feature(s): ",)
    for f in features:
        print("'" + featurenames[f] + "'" + " ",)
    print
    print("Errors: " + str(errors) + "/" + str(nt))# Now test the performance of a predictor based on a subset of features

In [6]:
test_model(mu, sigma, pi, [2], testx, testy)

Test error using feature(s): 
'Ash' 
Errors: 29/48


In [7]:
test_model(mu, sigma, pi, [0,2], testx, testy)

Test error using feature(s): 
'Alcohol' 
'Ash' 
Errors: 12/48


In [8]:
test_model(mu, sigma, pi, [0,2,6], testx, testy)

Test error using feature(s): 
'Alcohol' 
'Ash' 
'Flavanoids' 
Errors: 3/48


In [9]:
test_model(mu, sigma, pi, range(13), testx, testy)

Test error using feature(s): 
'Alcohol' 
'Malic acid' 
'Ash' 
'Alcalinity of ash' 
'Magnesium' 
'Total phenols' 
'Flavanoids' 
'Nonflavanoid phenols' 
'Proanthocyanins' 
'Color intensity' 
'Hue' 
'OD280/OD315 of diluted wines' 
'Proline' 
Errors: 2/48
